In [35]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
import numpy as np
conn = sqlite3.connect('pitchRx1314.sqlite3')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)


from hmmlearn.hmm import MultinomialHMM
from hmmlearn.hmm import GaussianHMM
# from pomegranate import *

from sklearn.preprocessing import normalize

from itertools import groupby
from bokeh.plotting import show, figure
from bokeh.charts import Scatter
from bokeh.io import output_notebook

import plotly.plotly as py
import plotly.graph_objs as go
# %load_ext rpy2.ipython
# %load_ext pymatbridge


In [2]:
def get_split_index(df):
        count = 0
        i = 0
        for i in range(len(df) - 1):
            batter_old = df.batter[i]
            batter_new = df.batter[i + 1]
            if batter_old != batter_new:
                count += 1
            if count == 18:
                break
        return (i + 1)

def split_3rd(df):
    before_3rd = pd.DataFrame()
    after_3rd = pd.DataFrame()
    dates = df.date.unique()
    for d in dates:
        game_df = df[df.date == d].reset_index(drop = True)
        split_index = get_split_index(game_df) # Get row number that splits third time through order
        before_3rd = before_3rd.append(game_df.iloc[0:split_index,:]) # Append dataframes
        after_3rd = after_3rd.append(game_df.iloc[split_index:,:])
        
    return before_3rd, after_3rd.reset_index(drop = True)


def innings_pitched(df):
    one_out_plays = ['Groundout','Strikeout','Lineout','Sac Bunt','Pop Out','Flyout','Sac Fly','Forceout',
                     'Runner Out','Fielders Choice Out','Bunt Groundout','Fielders Choice','Bunt Pop Out','Sac Fly',
                    'Bunt Lineout']
    two_out_plays = ['Double Play','Grounded Into DP','Strikeout - DP','Sac Fly DP','Sacrifice Bunt DP']
    three_out_plays = ['Triple Play']
    dates = list(df.date.unique())
    ip = 0
    for d in dates:
        max_inning = max(df[df.date == d].inning).astype(int)
        game_ip = 0
        for i in range(1, (max_inning + 1)) :
            df1 = df[(df.date == d) & (df.inning == i)]
            one_outs = sum(df1.event.isin(one_out_plays))
            two_outs = 2 * sum(df1.event.isin(two_out_plays))
            three_outs = 3 * sum(df1.event.isin(three_out_plays))
            game_ip += (one_outs + two_outs + three_outs)/3.0
            ip += (one_outs + two_outs + three_outs)/3.0
    return ip

def strikeout_ratio(df):
    return 9.0 * sum(df.event.isin(['Strikeout','Strikeout - DP']))/innings_pitched(df)

def fip(df):
    hr = sum(df.event.isin(['Home Run']))
    bb = sum(df.event.isin(['Walk']))
    hbp = sum(df.event.isin(['Hit By Pitch']))
    k = sum(df.event.isin(['Strikeout','Strikeout - DP']))
    ip = innings_pitched(df)
    return (13 * hr + 3 * (bb + hbp) - 2*k)/ip + 3.10

def whip(df):
    bb = sum(df.event.isin(['Walk']))
    hits = sum(df.event.isin(['Single','Home Run','Double','Triple']))
    return (bb + hits)/innings_pitched(df)

def walks_ratio(df):
    return sum(df.event.isin(['Walk']))/innings_pitched(df) * 9.0


In [3]:
pitchers = list(pd.read_csv('pitchers.csv')['name'])

before_fip = []
after_fip = []
before_whip = []
after_whip = []
before_strikeout = []
after_strikeout = []
before_walks = []
after_walks = []
pitch = []
for pitcher in pitchers:
#     print pitcher
    sql_query = '''
    SELECT *
    FROM atbat
    WHERE pitcher_name == ? AND (date >  '2014_03_30' AND date < '2014_09_30' )
    '''
    player_bat_df = pd.read_sql_query(sql_query, con = conn, params = [pitcher])
    if len(player_bat_df) == 0:
        continue

    player_bat_df = player_bat_df.sort_values(['date', 'num']).reset_index(drop = True)
    
    before,after = split_3rd(player_bat_df)
    
    before_fip.append(fip(before))
    after_fip.append(fip(after))
    before_whip.append(whip(before))
    after_whip.append(whip(after))
    before_strikeout.append(strikeout_ratio(before))
    after_strikeout.append(strikeout_ratio(after))
    before_walks.append(walks_ratio(before))
    after_walks.append(walks_ratio(after))
    pitch.append(pitcher)
    
stats_df = pd.DataFrame({'aaname':pitch,
                        'before_fip':before_fip,
                        'after_fip':after_fip,
                       'before_whip':before_whip,
                       'after_whip':after_whip,
                       'before_strikeout':before_strikeout,
                       'after_strikeout':after_strikeout,
                       'before_walks':before_walks,
                       'after_walks':after_walks})
                            

In [4]:
cols = ['aaname','before_fip','after_fip','before_whip','after_whip',
       'before_strikeout','after_strikeout','before_walks','after_walks']

stats_df[cols].head()

,aaname,before_fip,after_fip,before_whip,after_whip,before_strikeout,after_strikeout,before_walks,after_walks
0,C.C. Sabathia,5.089796,3.904878,1.346939,1.756098,8.540816,11.195122,2.479592,0.658537
1,Clayton Kershaw,1.579675,2.124631,0.845528,0.886700,11.048780,10.906404,1.390244,1.463054
2,David Price,2.726697,2.741196,1.160633,0.956811,9.895928,9.777409,1.466063,1.166113
3,Justin Verlander,3.776923,3.653398,1.261538,1.616505,6.369231,7.995146,2.700000,2.883495
4,R.A. Dickey,3.679775,5.584848,1.105618,1.484848,7.644944,6.409091,2.730337,3.681818


In [5]:
len(stats_df[stats_df.before_fip < stats_df.after_fip])/float(len(stats_df))

0.6533333333333333

In [6]:
len(stats_df[stats_df.before_whip < stats_df.after_whip])/float(len(stats_df))

0.6266666666666667

In [7]:
len(stats_df[stats_df.before_strikeout > stats_df.after_strikeout])/float(len(stats_df))

0.72

In [8]:
len(stats_df[stats_df.before_walks < stats_df.after_walks])/float(len(stats_df))

0.5333333333333333

In [9]:
(stats_df.before_strikeout - stats_df.after_strikeout).sort_values()

65   -2.852871e+00
0    -2.654306e+00
71   -2.166921e+00
43   -1.913485e+00
3    -1.625915e+00
53   -1.541791e+00
38   -1.361345e+00
25   -1.237316e+00
29   -1.198631e+00
57   -9.627907e-01
60   -9.209976e-01
41   -9.202063e-01
20   -8.557615e-01
19   -8.115950e-01
51   -6.731456e-01
15   -6.380345e-01
69   -1.145889e-01
5    -1.050468e-01
27   -5.414843e-02
40   -7.105427e-15
42    0.000000e+00
8     3.339123e-02
2     1.185190e-01
22    1.258741e-01
1     1.423765e-01
49    2.552521e-01
31    3.120567e-01
66    3.947214e-01
62    4.207602e-01
44    4.372826e-01
37    4.534022e-01
26    4.558983e-01
30    5.181346e-01
35    5.718974e-01
18    6.583703e-01
24    6.594470e-01
72    6.630027e-01
34    7.180779e-01
10    7.473383e-01
28    7.511459e-01
12    7.736037e-01
61    7.773541e-01
52    7.915593e-01
9     8.517881e-01
59    8.952007e-01
48    9.172681e-01
7     9.937500e-01
33    1.023061e+00
23    1.046108e+00
55    1.159274e+00
32    1.197233e+00
4     1.235853e+00
6     1.2766

In [10]:
# stats_df.aaname[abs(stats_df.before_strikeout - stats_df.after_strikeout).argsort()[len(stats_df)]]
stats_df.aaname[67]

'Jon Lester'

In [11]:
sql_query = '''
SELECT 
X.date as date,
X.event as event,
pitch.inning as inning,
X.o as o,
X.s as o,
X.b as b, 
X.s as s, 
X.pitcher_name as pitcher_name,
X.batter_name as batter,
pitch.type as type,
pitch.x0 as x_location,
pitch.z0 as z_location
FROM (
    SELECT *
    FROM atbat
    WHERE pitcher_name = ? AND (date >  '2014_03_30' AND date < '2014_09_30')
) AS X
INNER JOIN pitch on X.gameday_link = pitch.gameday_link AND X.num = pitch.num
ORDER BY date,inning, id, o ASC
'''
pitcher = 'Jered Weaver'
player_df = pd.read_sql_query(sql_query, con = conn, params = [pitcher])

In [31]:
df.head()

,Unnamed: 0,date,event,inning,o,o.1,b,s,pitcher_name,batter,type,x_location,z_location
0,0,2014_03_31,Single,1.0,0.0,2.0,3.0,2.0,Jered Weaver,Abraham Almonte,S,-2.588,6.037
1,1,2014_03_31,Single,1.0,0.0,2.0,3.0,2.0,Jered Weaver,Abraham Almonte,S,-2.655,5.939
2,2,2014_03_31,Single,1.0,0.0,2.0,3.0,2.0,Jered Weaver,Abraham Almonte,B,-2.781,5.963
3,3,2014_03_31,Single,1.0,0.0,2.0,3.0,2.0,Jered Weaver,Abraham Almonte,S,-2.668,5.848
4,4,2014_03_31,Single,1.0,0.0,2.0,3.0,2.0,Jered Weaver,Abraham Almonte,B,-2.492,5.760


In [38]:
trace = go.Scatter(
    x = player_df.x_location,
    y = player_df.z_location,
    mode = 'markers'
)

data = [trace]
py.iplot(data, filename = 'pitches')


In [32]:
stats_df[stats_df.aaname == 'Jered Weaver'][cols]

,aaname,before_fip,after_fip,before_whip,after_whip,before_strikeout,after_strikeout,before_walks,after_walks
7,Jered Weaver,3.799519,4.666667,1.088942,1.483333,7.59375,6.6,2.53125,3.45


In [33]:
dates = list(player_df.date.unique())

In [34]:
# dates = list(player_df.date.unqiue())
# sequences = np.array((player_df[player_df.date == dates[0]].type == 'S') * 1).reshape(len(player_df[player_df.date == dates[0]]), 1)
# lengths = [len(sequences)]
# for d in list(player_df.date.unique())[1:3]:
#     seq = np.array((player_df[player_df.date == d].type == 'S') * 1).reshape(len(player_df[player_df.date == d]), 1)
#     sequences = np.concatenate([sequences, seq]) 
#     lengths.append(len(seq))
j = 0
sequences = np.array((player_df[player_df.date == dates[j]].type == 'S') * 1).reshape(len(player_df[player_df.date == dates[j]]), 1)
# sequences = np.array((player_df[player_df.date == dates[j]].type == 'S') * 1)
lengths = [len(sequences)]

In [99]:
# for j in range(len(player_df.date.unique())):
for j in range(1):

    sequences = np.array((player_df[player_df.date == dates[j]].type == 'S') * 1).reshape(len(player_df[player_df.date == dates[j]]), 1)
    lengths = [len(sequences)]
    hmm = MultinomialHMM(n_components = 3)
    hmm.fit(sequences)
    hmm.predict(sequences)
    print ''
    print hmm.transmat_
    print ''
    print hmm.startprob_
    print ''
    print hmm.emissionprob_

    hmmdf = pd.DataFrame({'states':hmm.predict(sequences),
                          'event':player_df[player_df.date == dates[j]].type,
                          'date':player_df[player_df.date == dates[j]].date,
                          'batter':player_df[player_df.date == dates[j]].batter_name, 
                          'event':player_df[player_df.date == dates[j]].event,
                          'inning':player_df[player_df.date == dates[j]].inning
                         }).reset_index()

    before, after = split_3rd(hmmdf)
    print ''
    print before.states.value_counts()
    print ''
    print after.states.value_counts()
    print ''
    print 'fip before', fip(before)
    print ''
    print 'fip after', fip(after)


[[ 0.42251964  0.20835775  0.36912261]
 [ 0.28205842  0.41709086  0.30085072]
 [ 0.40908447  0.22760384  0.36331169]]

[  8.62504768e-10   9.99996326e-01   3.67327312e-06]

[[ 0.89152613  0.10847387]
 [ 0.0550439   0.9449561 ]
 [ 0.72994495  0.27005505]]

0    38
1    31
Name: states, dtype: int64

0    19
1     7
Name: states, dtype: int64

fip before 1.01304347826

fip after 12.1


In [108]:
states = hmmdf.states

grouped_states = [(k, sum(1 for i in g)) for k,g in groupby(states)]

groupby(states)

In [106]:
grouped_states

[(1, 2),
 (0, 1),
 (1, 1),
 (0, 5),
 (1, 1),
 (0, 3),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 4),
 (1, 2),
 (0, 1),
 (1, 1),
 (0, 1),
 (1, 2),
 (0, 1),
 (1, 2),
 (0, 8),
 (1, 4),
 (0, 3),
 (1, 1),
 (0, 3),
 (1, 7),
 (0, 2),
 (1, 3),
 (0, 4),
 (1, 2),
 (0, 1),
 (1, 2),
 (0, 2),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 2),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 3),
 (1, 1),
 (0, 5),
 (1, 1),
 (0, 5)]

In [109]:
hmmdf

,index,batter,date,event,inning,states
0,0,Abraham Almonte,2014_03_31,Single,1.0,1
1,1,Abraham Almonte,2014_03_31,Single,1.0,1
2,2,Abraham Almonte,2014_03_31,Single,1.0,0
3,3,Abraham Almonte,2014_03_31,Single,1.0,1
4,4,Abraham Almonte,2014_03_31,Single,1.0,0
5,5,Abraham Almonte,2014_03_31,Single,1.0,0
6,6,Abraham Almonte,2014_03_31,Single,1.0,0
7,7,Brad Miller,2014_03_31,Pop Out,1.0,0
8,8,Brad Miller,2014_03_31,Pop Out,1.0,0
9,9,Brad Miller,2014_03_31,Pop Out,1.0,1
